In [1]:
from tablecloth import queryspace

A `QuerySpace` handles the grouping of query elements.

In [51]:
qs = queryspace.QuerySpace()

A `QueryTemplate` is a template for a query, or a subquery. Placeholders can be identified with `qs` (for `QuerySpace`), a dot, and the name of an other query element in the query space.

In [52]:
qs['count_users'] = queryspace.QueryTemplate(
    """
    SELECT
      COUNT(DISTINCT uid) AS n
    FROM
      {qs.relevant_transactions}
    """
)

In [53]:
for k, v in qs.items():
    print(f'{k}: {type(v).__name__}')

count_users: QueryTemplate
relevant_transactions: Placeholder


The query space will not be able to make a sql when there remain unspecified placeholders.

In [55]:
try:
    qs.make('count_users')
except Exception as e:
    print(repr(e))

UnspecifiedPlaceholder('relevant_transactions',)


The placeholder can be specified when making the SQL query.

In [39]:
sql = qs.make('count_users',
              relevant_transactions=queryspace.TableName('transactions'))
print(sql)


    SELECT
      COUNT(DISTINCT uid) AS n
    FROM
      transactions
    


A query space can handle several query template, and work the dependencies from placeholders in the templates.

In [42]:
relevant_transactions = queryspace.QueryTemplate(
    """
    SELECT
      *
    FROM
      {qs.transactions_table}
    """
)

In [56]:
try:
    qs.make('count_users', relevant_transactions=relevant_transactions)
except Exception as e:
    print(repr(e))

NameNotFoundError('transactions_table',)


In [45]:
qs['relevant_transactions'] = relevant_transactions

In [57]:
try:
    qs.make('count_users')
except Exception as e:
    print(repr(e))

UnspecifiedPlaceholder('relevant_transactions',)


In [49]:
sql = qs.make('count_users', transactions_table=queryspace.TableName('transactions'))
print(sql)

WITH
relevant_transactions AS (
    SELECT
      *
    FROM
      transactions
    )

    SELECT
      COUNT(DISTINCT uid) AS n
    FROM
      relevant_transactions
    
